In [41]:
# IMPORTS of Libraires needed for project
from spacy.lang.en.stop_words import STOP_WORDS

from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import decomposition

from collections import Counter
from collections import OrderedDict

from geopy.geocoders import Nominatim

from operator import itemgetter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopy.distance

import requests
import json
import random
import nltk
import difflib
import spacy
import re, string

# This tokenizer divides a text into a list of sentences by using
# an unsupervised algorithm to build a model for abbreviation words, collocations, and words.
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('wordnet')

pd.options.mode.chained_assignment = None

print('OK')

OK


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [42]:
# Check the version of Libraries
print(f"Pandas version: {pd.__version__}")
print(f"Numpy version: {np.__version__}")

Pandas version: 1.3.5
Numpy version: 1.21.6


In [43]:
# Declaring Main Tags 
# In production these will be accessed through request to the server
tags = [
	"skin", "haircut", "makeup", "colouring", "color", "colour", "yoga", 
	"nutrition", "fitness", "tanning", "piercing", "tattoo", "aesthetics", "spa",
	"barber", "eyelash", "eyelashes", "eyebrow", "eyebrows", "wellness", "body",
	"massage", "hair", "nail", "nails", "face", "product", "health", "hand",
	"general"
]

In [44]:
# Loading Artist Data as Pandas DataFrame
ds = pd.read_json('/content/Artist_200.json')

# Adding required collumns to dataset 
ds.insert(0, 'activity', 0)
ds.insert(0, 'price', 0)


In [45]:
# Function to get Artist Price
# In Production this will be request to the server 
def getPrice(artist):

  # if artis declered prices of the services the price will be calculated
  # As the dataset does not include this data, its being generated randomly

  if artist['price_range']['lowest'] is None:
    return random.randint(10,150)
  else:
    return (int(artist['price_range']['highest']) + int(artist['price_range']['lowest'])) / 2
  
  return 

# Curently Generating Random number to simulate Artist activity on the platform
# These would be otherwise accuired throught requests to the server

for index, artist in ds.iterrows():

  ds['longitude'][index] = random.uniform(0, -2)
  ds['latitude'][index] = random.uniform(49, 51)
  ds['price'][index] = getPrice(artist)
  if not artist['collection_tags']:
    ds['collection_tags'][index] = random.choices(tags, k=3)
  ds['activity'][index] = random.randint(0,7)
  if ds['occupation'][index] is None:
    ds['occupation'][index] = "beautician"

# Combining required Features fro ML model
combined_Feature = [ds['user_id'],
                    ds['latitude'].apply(str),
                    ds['longitude'].apply(str),
                    ds['occupation'].apply(str),
                    ds['collection_tags'],
                    ds['activity'],
                    ds['price']]

# Declering generated data as Data Frame
artist_data = pd.DataFrame(combined_Feature).transpose()

print('OK')


OK


In [46]:
# Artist Data visualization after Pre-Procesing
artist_data.head()

,user_id,latitude,longitude,occupation,collection_tags,activity,price
0,ba64dbbd-bfa6-4d8f-bf41-ee832f0de4fd,49.75029480849041,-0.47545262309245384,Speech Pathologist,"[piercing, hand, fitness]",0,80
1,f78970f6-0d0f-4717-97f3-f8886a8201c9,50.11610467253455,-1.652552712625605,Operator,"[eyebrow, fitness, nails]",2,55
2,a8a6aa5f-d774-435b-b006-19c5c211b86b,49.99650361421796,-1.7732817505840188,Nail Professional,"[Nails, Face, Hair, Hair Removal, Body, Massag...",0,34
3,a51d1e71-d715-4772-b27a-f7f804de948a,49.00486254485072,-1.890873108166399,Makeup Artist,"[Face, Nails, Hair, Hair Removal, Body, Massag...",6,74
4,e1302c2c-52e8-413c-8681-3094d691f150,50.89528856437689,-0.5146401732846184,Staff Scientist,"[colour, fitness, tattoo]",3,35


In [47]:
# Stemmer and Lemmatizer Activation 
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
nlp = spacy.load("en_core_web_sm")

In [48]:
# Loading Questions Data as Pandas DataFrame

questions = pd.read_json("/content/Questions_600.json")

# Adding required collumns to dataset 

questions.insert(0, 'longitude', 0.0)
questions.insert(0, 'latitude', 0.0)
questions.insert(0, 'user_id', 0)
questions.insert(0, 'occupation', 0)
questions.insert(0, 'category', "")
questions.insert(0, 'distance', 0.0)
questions.insert(0, 'best_artist', "")
print("OK")

OK


In [49]:
# Function to obtain user Occupation
# As these are dummy data the random occupation is generated for each user from 
# the list of occupations

def getUserOccupation(userId):

  occupation = random.choice(list(ds['occupation']))

  return occupation

# Function to obtain tokenized data

def tokenize(text):
  tokens = [word for word in nltk.word_tokenize(text) if (len(word) > 3 and len(word.strip('Xx/')) > 2)]
  return tokens

# DATA Preprocesing 

# for loop iterrate over questions dataset to genaret random values 
# these values would be otherwise filled by different requests to the server
for index, question in questions.iterrows():
  questions['user_id'][index] = questions['user_details'][index]['user_id']
  questions['occupation'][index] = getUserOccupation("userId")
  questions['longitude'][index] = round(random.uniform(1, -4), 8)
  questions['latitude'][index] = round(random.uniform(48, 52), 8)

  if not question['question_description']:
    questions['question_description'][index] = " "
  elif question['question_description'] is None:
    questions['question_description'][index] = " "
  else:
    pass

  # combining question header with question description into one string
  questionContent = question['question_header'] + ' , ' + str(question['question_description'])

  # removing unnecessary punctuation
  chars = '[%s]+' % re.escape(string.punctuation)
  questionContent = re.sub(chars, '', questionContent)

  # Lemmatization of each word in question
  doc = [lemmatizer.lemmatize(wd.lower()) for wd in questionContent.split()]

  # removing unnecessary word from the corpus
  for word in doc:
    if word in STOP_WORDS:
      doc.remove(word)
    if word == "  ":
      doc.remove(word)

  # declaring question category if any word within corpus match tags 
  # its being stored in the category list
  for word in doc:
    category = []
    if word in tags:
      category.append(word)

    # if the cotegory list is not empty the question category is equal to 
    # the content of the list
    if len(category) > 0:
      questions['category'][index] = category
    # otherwise is equal to general
    else:
      questions['category'][index] = ['general']

# declaring train test datasets
train, test = [], []

# if the question category is general the test data will be appended
for index, question in questions.iterrows():
  if question['category'] != ['general']:
    train.append(question)
  # otherwise train data will be appended 
  else:
    test.append(question)

# These data will be converted into pandas dataframe
train = pd.DataFrame.from_records(train)
test = pd.DataFrame.from_records(test)

# Confirming the volume of the datasets
print(f"train: {len(test)} , test: {len(train)}")

# question content being created in Both datasets from question header and the question description
train['question_content'] = train['question_header'] + ',' + train['question_description']
train_df = train[['question_content', 'question_id', 'anonymous', 'category', 'user_id', 'occupation', 'longitude', 'latitude', 'distance', 'best_artist']]

test['question_content'] = test['question_header'] + ',' + test['question_description']
test_df = test[['question_content', 'question_id', 'anonymous', 'user_id', 'occupation', 'longitude', 'latitude', 'distance', 'best_artist']]

# vectorizer being declared
vectorizer_tf = TfidfVectorizer(tokenizer=tokenize, stop_words='english', max_df=0.75, max_features=10000, use_idf=False, norm=None)

# fiting the vectorizer with training data
tf_vectors = vectorizer_tf.fit_transform(train_df.question_content)

# Lda model being declared
lda = decomposition.LatentDirichletAllocation(n_components=len(tags), max_iter=15, learning_method='online', learning_offset=70, n_jobs=-1, random_state=25)

# vectorised data are fited into LDA model
W1 = lda.fit_transform(tf_vectors)
H1 = lda.components_

# declaring number of words for LDA model
num_words = 8

# extractin genareted topics from LDA model for each question
vocab = np.array(vectorizer_tf.get_feature_names_out())
top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_words-1:-1]]
topic_word = ([top_words(t) for t in H1])
topics = [' '.join(t) for t in topic_word]

# Genareting significant topic for each question in train data
colnames = [tags[i] for i in range(lda.n_components)]
docnames = ["Question " + str(i) for i in range(len(train_df.question_content))]
df_doc_topic = pd.DataFrame(np.round(W1, 2), columns=colnames, index=docnames)
significant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic['dominant_topic'] = significant_topic

WTest = lda.transform(vectorizer_tf.transform(test_df.question_content))

# Genareting significant topic for each question in test data
colnames = [tags[i] for i in range(lda.n_components)]
docnames = ["Question " + str(i) for i in range(len(test_df.question_content))]
df_doc_topic_test = pd.DataFrame(np.round(WTest, 2), columns=colnames, index=docnames)
significant_topic = np.argmax(df_doc_topic_test.values, axis=1)
df_doc_topic_test['dominant_topic'] = significant_topic

lda_components = lda.components_
terms = vectorizer_tf.get_feature_names_out()

# Declaring Topics list 
Topics = []

# Extracting Topics based on the words representing it from the list of Tags
for index, component in enumerate(lda_components):
    zipped = zip(terms, component)
    top_terms_key=sorted(zipped, key = lambda t: t[1], reverse=True)[:6]
    top_terms_list=list(dict(top_terms_key).keys())
    category = []

    for word in top_terms_list:
      if word in tags:
        category.append(word)
      word = lemmatizer.lemmatize(word.lower())
      if word in tags:
        category.append(word)
    if len(category) > 0:
      category = list(dict.fromkeys(category))
    else:
      category = "general"
    Topics.append(category)
# declering a list with clasified data 
clasifiedData = []

# iteration over test data to set generated category as
# question category
for index, row in test_df.iterrows():
  category = Topics[df_doc_topic_test['dominant_topic'][index]]
  row['category'] = category
  clasifiedData.append(row)

# converting list of clasified data into pandas dataframe
clasifiedData = pd.DataFrame(clasifiedData)

# merging train data with clasified data
frames = [train_df, clasifiedData]
questionsResult = pd.concat(frames, ignore_index=True)

print("ok")

train: 476 , test: 124
ok


In [50]:
# Log Likelyhood: Higher the better
print(f"Log Likelihood: {lda.score(tf_vectors)}")

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print(f"Perplexity: {lda.perplexity(tf_vectors)}")

# See model parameters
lda.get_params()

Log Likelihood: -14414.826060276766
Perplexity: 947861783231118.8


{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 70,
 'max_doc_update_iter': 100,
 'max_iter': 15,
 'mean_change_tol': 0.001,
 'n_components': 30,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 25,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}

In [51]:
# printing head of topic metrix
df_doc_topic.head()

,skin,haircut,makeup,colouring,color,colour,yoga,nutrition,fitness,tanning,...,massage,hair,nail,nails,face,product,health,hand,general,dominant_topic
Question 0,0.00,0.00,0.00,0.00,0.00,0.00,0.86,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,6
Question 1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.86,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,9
Question 2,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,...,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,16
Question 3,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,...,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.68,29
Question 4,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,...,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,17


In [52]:
# printing result of LDA model 
questionsResult.head()

,question_content,question_id,anonymous,category,user_id,occupation,longitude,latitude,distance,best_artist
0,Have you ever experienced any allergic reactio...,335623e5-956f-4ce6-99c3-55b635bd54f6,False,[product],c0732824-a076-428a-9898-ab67cbac1172,VP Sales,-2.491144,50.964859,0.0,
1,DO GEL NAILS HELP IF YOU ARE A NAIL-BITER AND/...,ac75d1ea-66e3-4ca7-be22-c9b6b0a3edb3,False,[nail],1a07bd99-b7fd-4a29-b651-231f61868c84,Salon/Spa Owner,-1.589721,48.922424,0.0,
2,"I have a health condition. Can I still do yoga?,",9390dad0-ab50-4a13-b641-849ab60b5a9c,False,[yoga],63167940-0feb-4378-a605-553c94a68090,Barber,-3.090505,49.193809,0.0,
3,"Can you dye over jet black hair?,",85116450-e502-4227-ad0b-440dcac068cc,False,[hair],7a2e2a94-d6f9-4a1e-a452-61cb4e79e8ec,Makeup Artist,-3.540601,50.332758,0.0,
4,Which do you think is the most dangerous for y...,f6e0d200-7c72-4bfe-ab9d-1da604d65bc5,False,[health],f8ed43ce-f569-4182-a60e-03e0597c9939,Assistant Professor,-2.977108,49.732216,0.0,


In [53]:
count = 0

# Function to get best artist
def getBestArtist(question, data):
  # List to store filtered data
  artistLocationMatches = []
  artistLocationFiltered = []
  artistTagsMatches = []
  artistPriceMatches = []

  # Function which calculates the distance between user and artist

  def getDistance(userLatitude, userLongitude, artistLatitude, artistLongitude):
    return geopy.distance.geodesic((userLatitude, userLongitude), (artistLatitude, artistLongitude)).km

  # Filter artist based on the tag maching the question category

  for index, artist in data.iterrows():
    for i in range(len(artist['collection_tags'])):
      artist['collection_tags'][i] = artist['collection_tags'][i].lower()

    for category in question['category']:
      if category in artist['collection_tags'][i]:
        artistTagsMatches.append(artist)
        break
  
  # storing user data with calulated distance in the list

  for index, artist in enumerate(artistTagsMatches):
    distance = getDistance(question['latitude'], question['longitude'], artist['latitude'], artist['longitude'])
    userIndex = np.where(data['user_id'] == artist['user_id'])
    artistLocationMatches.append([distance, userIndex[0][0]])

  # If location is in radius of 100 km artist will be moved to futher filtering

  SortedArtistLocation = sorted(artistLocationMatches,
                                key=itemgetter(0))   

  SortedArtistLocation = SortedArtistLocation[:10]

  # Futher filtering to find 3 artist with best price range

  for artist in SortedArtistLocation:
    artistPriceMatches.append([data['price'][artist[1]], artist[1]])

  PriceMatches = sorted(artistPriceMatches,
                                key=itemgetter(0))

  PriceMatches = PriceMatches[-3:]

  # Chousing the most active artist

  ArtistActivity = []

  for artist in PriceMatches:
    ArtistActivity.append([data['activity'][artist[1]], artist[1]])

  # list of most active artis being sorted
  MostActive = sorted(ArtistActivity,
                      reverse=True,
                      key=itemgetter(0))
  
  # if list has artist in it the best artis is chosen
  if len(MostActive) > 0:
    response = data['user_id'][MostActive[0][1]]
  # otherwise to avoid error the user being selected automaticly
  else:
    response = data['user_id'][1]
    count =+ 1

  return response

# iteration over question dat to set best artist
for index, question in questionsResult.iterrows():
  questionsResult.at[index, 'best_artist'] = getBestArtist(question, artist_data)

# converting questions result into pandas dataframe 
questions_data = pd.DataFrame(questionsResult)

print("ok")

ok


*italicised text*

In [54]:
# Visualisation of Question data after pre-procesing
questions_data.head()

,question_content,question_id,anonymous,category,user_id,occupation,longitude,latitude,distance,best_artist
0,Have you ever experienced any allergic reactio...,335623e5-956f-4ce6-99c3-55b635bd54f6,False,[product],c0732824-a076-428a-9898-ab67cbac1172,VP Sales,-2.491144,50.964859,0.0,f80d6d86-cdcd-4b35-9844-3aa203bdf7ef
1,DO GEL NAILS HELP IF YOU ARE A NAIL-BITER AND/...,ac75d1ea-66e3-4ca7-be22-c9b6b0a3edb3,False,[nail],1a07bd99-b7fd-4a29-b651-231f61868c84,Salon/Spa Owner,-1.589721,48.922424,0.0,fd896ac2-083a-4dcf-a6e0-b67a2e77a4f8
2,"I have a health condition. Can I still do yoga?,",9390dad0-ab50-4a13-b641-849ab60b5a9c,False,[yoga],63167940-0feb-4378-a605-553c94a68090,Barber,-3.090505,49.193809,0.0,1d01e207-d038-46a9-97c5-ee32864191ed
3,"Can you dye over jet black hair?,",85116450-e502-4227-ad0b-440dcac068cc,False,[hair],7a2e2a94-d6f9-4a1e-a452-61cb4e79e8ec,Makeup Artist,-3.540601,50.332758,0.0,29b8a8de-1b84-49aa-87e2-ad905e008614
4,Which do you think is the most dangerous for y...,f6e0d200-7c72-4bfe-ab9d-1da604d65bc5,False,[health],f8ed43ce-f569-4182-a60e-03e0597c9939,Assistant Professor,-2.977108,49.732216,0.0,ff1983f8-619e-440c-b5d7-7a119219dac8


In [62]:
# declering TFid Vectorizer
vectorizer = TfidfVectorizer()

# renaming artist data set
artist_d = artist_data

# Spliting dataset
X = questions_data.drop('best_artist', axis=1)
y = questions_data['best_artist']

X.head()

,question_content,question_id,anonymous,category,user_id,occupation,longitude,latitude,distance
0,Have you ever experienced any allergic reactio...,335623e5-956f-4ce6-99c3-55b635bd54f6,False,[product],c0732824-a076-428a-9898-ab67cbac1172,VP Sales,-2.491144,50.964859,0.0
1,DO GEL NAILS HELP IF YOU ARE A NAIL-BITER AND/...,ac75d1ea-66e3-4ca7-be22-c9b6b0a3edb3,False,[nail],1a07bd99-b7fd-4a29-b651-231f61868c84,Salon/Spa Owner,-1.589721,48.922424,0.0
2,"I have a health condition. Can I still do yoga?,",9390dad0-ab50-4a13-b641-849ab60b5a9c,False,[yoga],63167940-0feb-4378-a605-553c94a68090,Barber,-3.090505,49.193809,0.0
3,"Can you dye over jet black hair?,",85116450-e502-4227-ad0b-440dcac068cc,False,[hair],7a2e2a94-d6f9-4a1e-a452-61cb4e79e8ec,Makeup Artist,-3.540601,50.332758,0.0
4,Which do you think is the most dangerous for y...,f6e0d200-7c72-4bfe-ab9d-1da604d65bc5,False,[health],f8ed43ce-f569-4182-a60e-03e0597c9939,Assistant Professor,-2.977108,49.732216,0.0


In [63]:
# converting X data into suitable format for machine learning model
X = (X['question_id'].astype("string") +
     X['category'].astype("string") +
     X['user_id'].astype("string") +
     X['longitude'].astype("string") +
     X['latitude'].astype("string") +
     X['question_content'].astype("string")
    )

# vectorizing X data 
X = vectorizer.fit_transform(X)

# spliting dataset into test and train data
X_train, X_test, y_train, y_test = train_test_split(
                                            X,
                                            y,
                                            test_size=0.15,
                                            random_state=48
                                            )

print('Ok')

Ok


In [64]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=120)

#Train the model using the training sets 
clf.fit(X_train,y_train)

# Test model with Prediction
y_pred=clf.predict(X_test)

In [65]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.5111111111111111


In [66]:
# FUnction to get user address
def getAddress(lat, long):
  location = geolocator.reverse(lat, long)
  return location

# converting artist data into string for vectorizer
V_artist = artist_data['user_id'] + " , " + artist_data['latitude'] + " , " + artist_data['longitude'] + " , " + artist_data['occupation'] + " , " + artist_data['activity'].astype('string') + " , " + artist_data['price'].astype('string')

# fiting the vectorizer
artist_vectors = vectorizer.fit_transform(V_artist) 

# fiting cosine similarity with vectorized data
similarity = cosine_similarity(artist_vectors)

# extracting user ID's from list of artist.
list_of_all_artist = artist_data['user_id'].tolist()

# declaring geolocator
geolocator = Nominatim(user_agent="RecomenderSystem", timeout=4)

y = 1

# iteration over predicted data
for prediction in y_pred:
  # finding predicted artist in the list of all artist
  find_close_match = difflib.get_close_matches(prediction, list_of_all_artist)
  # extracting the artist id
  close_match = find_close_match[0]
  # finding artis data based on the extracted id
  single_artist = artist_data[artist_data.user_id == close_match]
  # generating similarity score for the artis data
  similarity_score = list(enumerate(similarity[single_artist.index.values[0]]))
  # sorting similarity score
  sorted_similar_artist = sorted(similarity_score, key = lambda x:x[1], reverse = True)
  # extracting first three artists from the list
  sorted_similar_artist = sorted_similar_artist[:3]
  print(f"\nSuggested Artist for question: {y},\nContent: {questions_data['question_content'][y]} \n")

  i = 1
  y+=1
  # iterating over three artis to generate artist data
  for artist in sorted_similar_artist:
    index = artist[0]
    id_from_index = artist_d[artist_d.index==index]['user_id'].values[0]
    user = np.where(ds.user_id == id_from_index)
    # generating location of the artis 
    location = getAddress(ds.iloc[user[0][0]]['latitude'], ds.iloc[user[0][0]]['longitude'])
    if (i<4):
      # printing artis data 
      print(i, '.', f"Artist: {ds.iloc[user[0][0]]['display_name']}, \n    Specialisation: {str(ds.iloc[user[0][0]]['collection_tags'])[1:-1]},\n    Location: {location.address}, \n    Average Price: {artist_data.iloc[user[0][0]]['price']} \n")
      i+=1





Suggested Artist for question: 1,
Content: DO GEL NAILS HELP IF YOU ARE A NAIL-BITER AND/OR HAVE PROBLEM NAILS?,  



/usr/local/lib/python3.7/dist-packages/geopy/point.py:180: UserWarning: A single number has been passed to the Point constructor. This is probably a mistake, because constructing a Point with just a latitude seems senseless. If this is exactly what was meant, then pass the zero longitude explicitly to get rid of this warning.
  UserWarning)


1 . Artist: Jilleen, 
    Specialisation: 'nail', 'piercing', 'face',
    Location: France métropolitaine, France, 
    Average Price: 112 

2 . Artist: Dew, 
    Specialisation: 'body', 'color', 'color',
    Location: France métropolitaine, France, 
    Average Price: 54 

3 . Artist: Bea, 
    Specialisation: 'health', 'health', 'eyelash',
    Location: France métropolitaine, France, 
    Average Price: 22 


Suggested Artist for question: 2,
Content: I have a health condition. Can I still do yoga?,  

1 . Artist: Binky, 
    Specialisation: 'nails', 'haircut', 'makeup',
    Location: England, United Kingdom, 
    Average Price: 124 

2 . Artist: Lauritz, 
    Specialisation: 'colouring', 'health', 'hand',
    Location: D 511, Bretteville-sur-Dives, Saint-Pierre-en-Auge, Lisieux, Calvados, Normandie, France métropolitaine, 14170, France, 
    Average Price: 124 

3 . Artist: Torrance Salon, 
    Specialisation: 'face', 'wellness', 'nails', 'hair', 'hair removal', 'body', 'massage', '

In [67]:
print(f"First value refers to the artis itsef, following by most similar ones. \nSimilarity score: {sorted_similar_artist[:4]}")

First value refers to the artis itsef, following by most similar ones. 
Similarity score: [(142, 1.0), (41, 0.0895448174984234), (79, 0.011295433317216108)]
